In [1]:
def load_csv(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        csv_list = [row.strip().split(',') for row in f.readlines()]
    return csv_list

def printx(listx):
    for x in listx:
        print(x)

In [2]:
def parse_row(row):
    name = row[0].lower()
    prevElemCombos = row[1].lower().split(' / ')
    prevElemCombos = [p.lower().split('+') for p in prevElemCombos]
    if prevElemCombos == [['available from start.']]:
        prevElemCombos = [['BASE_ELEMENT']]
    prevElemCombos.sort()
    for p in prevElemCombos:
        p.sort()
    return [prevElemCombo + [name] for prevElemCombo in prevElemCombos]

def parse_rows(csv_list):
    r = []
    for row in csv_list:
        r.extend(parse_row(row))
    return r

In [3]:
elemsFilename = 'elements_data.csv'
csv_list = load_csv(elemsFilename)

In [4]:
import pandas as pd
from tqdm import tqdm

def create_elements_df(csv_list):
    r = parse_rows(csv_list)
    df = pd.DataFrame(r, columns=['elem1','elem2', 'result'])
    base_elements = get_base_elements(csv_list)
    return df[~df.index.isin(base_elements.index)]


def get_base_elements(csv_list):
    df = pd.DataFrame(parse_rows(csv_list), columns=['elem1','elem2', 'result'])
    return df[df['elem1']=='BASE_ELEMENT']

def get_elements_list(dfs):
    r = []
    for col in list(dfs.columns):
        r.extend(dfs[col].tolist())
    return list(set(r))


In [5]:
dfs = create_elements_df(csv_list)
dfs.head()

,elem1,elem2,result
0,rain,smog,acid rain
1,rain,smoke,acid rain
3,bird,metal,airplane
4,bird,steel,airplane
5,clock,sound,alarm clock


In [6]:
base_elements = get_base_elements(csv_list)['elem2'].tolist()
print(base_elements)

['air', 'earth', 'fire', 'water']


In [7]:
def get_unique_current_combos(curr_elements, dfs):
    possible_combos = [[elem1, elem2] for elem1 in curr_elements for elem2 in curr_elements if elem1 != elem2]
    for i in range(len(possible_combos)):
        possible_combos[i].sort()
        possible_combos[i] = '+'.join(possible_combos[i])
    possible_combos = list(set(possible_combos))
    possible_combos = [p.split('+') for p in possible_combos]
    return possible_combos

def get_elem_score(elem, dfs, levels=0):
    dFilter = (dfs['elem1']==elem) | (dfs['elem2']==elem)
    if levels == 0:
        return sum(dFilter)
    else:
        next_possible_elements = dfs.loc[dFilter, 'result'].tolist()
        return sum([get_elem_score(e, dfs, levels-1) for e in next_possible_elements])

def get_next_possible_combos(curr_elements, dfs):
    r = []
    unique_combos = get_unique_current_combos(curr_elements, dfs)
    for combo in unique_combos:
        l = (dfs['elem1']==combo[0]) & (dfs['elem2']==combo[1])
        if l.any():
            r.append([dfs.loc[l, 'elem1'].tolist()[0], dfs.loc[l, 'elem2'].tolist()[0] , dfs.loc[l, 'result'].tolist()[0]])
    return [x for x in r if x[-1] not in curr_elements]

def get_next_possible_elements(curr_elements, dfs):
    r = get_next_possible_combos(curr_elements, dfs)
    return [x[-1] for x in r]


In [8]:

def get_next_best_move(curr_elements, dfs, levels=0):
    next_possible_combos = get_next_possible_combos(curr_elements, dfs)
    scores = [get_elem_score(e[-1], dfs, levels) for e in next_possible_combos]
    return next_possible_combos[scores.index(max(scores))]

def fullCombo_to_string(combo):
    return ' + '.join(combo[:-1]) + ' = ' + combo[-1]

In [9]:
def get_n_moves(curr_elements, dfs, levels=0, n=10, print_moves=False):
    curr_elements = list(curr_elements)
    r = []
    for i in (range(n) if print_moves else tqdm(range(n))):
        newCombo = get_next_best_move(curr_elements, dfs, levels=levels)
        r.append(fullCombo_to_string(newCombo))
        if print_moves:
            print(i+1, fullCombo_to_string(newCombo))
        curr_elements.append(newCombo[-1])
        curr_elements.sort()
    return r

# **Complete example**


In [10]:
elemsFilename = 'elements_data.csv'
csv_list = load_csv(elemsFilename)

dfs = create_elements_df(csv_list)
base_elements = get_base_elements(csv_list)['elem2'].tolist()
nx = 50
x = get_n_moves(curr_elements=base_elements, dfs=dfs, levels=0, n=nx, print_moves=True)

1 air + fire = energy
2 air + energy = wind
3 air + water = rain
4 earth + rain = plant
5 earth + plant = grass
6 earth + fire = lava
7 air + lava = stone
8 fire + stone = metal
9 energy + metal = electricity
10 air + stone = sand
11 fire + sand = glass
12 metal + stone = blade
13 glass + metal = glasses
14 blade + metal = sword
15 electricity + metal = wire
16 earth + water = mud
17 mud + sand = clay
18 energy + water = steam
19 air + steam = cloud
20 air + cloud = sky
21 fire + sky = sun
22 rain + sun = rainbow
23 sky + stone = moon
24 moon + sky = night
25 night + sky = star
26 grass + sun = hay
27 star + sun = space
28 night + plant = carbon dioxide
29 metal + rainbow = gold
30 mud + plant = swamp
31 energy + swamp = life
32 earth + life = human
33 air + life = bird
34 human + metal = tool
35 life + stone = egg
36 bird + metal = airplane
37 human + plant = farmer
38 farmer + life = livestock
39 hay + livestock = horse
40 grass + livestock = cow
41 cow + farmer = milk
42 livestock +

In [14]:

x = get_n_moves(curr_elements=base_elements, dfs=dfs, levels=1, n=255, print_moves=True)

1 air + fire = energy
2 air + water = rain
3 earth + rain = plant
4 earth + plant = grass
5 air + energy = wind
6 earth + water = mud
7 energy + water = steam
8 air + steam = cloud
9 air + cloud = sky
10 fire + sky = sun
11 grass + mud = swamp
12 energy + swamp = life
13 earth + life = human
14 human + plant = farmer
15 farmer + life = livestock
16 air + life = bird
17 grass + livestock = cow
18 human + rain = cold
19 earth + fire = lava
20 air + lava = stone
21 fire + stone = metal
22 human + metal = tool
23 air + stone = sand
24 life + stone = egg
25 energy + metal = electricity
26 fire + sand = glass
27 metal + stone = blade
28 sun + tool = solar cell
29 sky + stone = moon
30 moon + sky = night
31 night + sky = star
32 egg + swamp = lizard
33 farmer + grass = hay
34 earth + tool = field
35 livestock + mud = pig
36 mud + sand = clay
37 cold + steam = snow
38 grass + plant = garden
39 cow + farmer = milk
40 egg + water = hard roe
41 egg + sand = turtle
42 electricity + glass = light b

In [12]:
x = get_n_moves(curr_elements=base_elements, dfs=dfs, levels=2, n=nx, print_moves=True)

1 air + fire = energy
2 air + water = rain
3 earth + rain = plant
4 earth + plant = grass
5 earth + fire = lava
6 air + lava = stone
7 fire + stone = metal
8 air + stone = sand
9 fire + sand = glass
10 energy + metal = electricity
11 earth + water = mud
12 grass + mud = swamp
13 energy + swamp = life
14 earth + life = human
15 human + metal = tool
16 human + plant = farmer
17 life + stone = egg
18 air + life = bird
19 egg + swamp = lizard
20 air + energy = wind
21 farmer + life = livestock
22 egg + sand = turtle
23 earth + tool = field
24 energy + water = steam
25 air + steam = cloud
26 air + cloud = sky
27 fire + sky = sun
28 sun + tool = solar cell
29 sky + stone = moon
30 moon + sky = night
31 metal + stone = blade
32 night + sky = star
33 sky + sun = day
34 mud + sun = brick
35 electricity + glass = light bulb
36 electricity + light bulb = light
37 grass + livestock = cow
38 farmer + field = wheat
39 light + tool = flashlight
40 blade + grass = scythe
41 earth + energy = earthquake

----------------------------------------